# VacationPy DAS
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os
import googlemaps
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv_file = "../3_output_data/cities.csv"

city_csv_file = pd.read_csv("../3_output_data/cities.csv")

city_csv_file.head()

,city_name,country,date,latitude,longitude,temp,humidity,cloudiness,wind_speed
0,Ribeira Grande,PT,1612267405,38.5167,-28.7000,57.20,67,20,23.02
1,Albany,US,1612267206,42.6001,-73.9662,26.01,86,100,1.01
2,Lüleburgaz,TR,1612267405,41.4039,27.3592,55.40,82,75,6.91
3,Kiryat Gat,IL,1612267406,31.6100,34.7642,78.01,41,0,3.00
4,Pevek,RU,1612267406,69.7008,170.3133,6.49,92,100,7.25


In [3]:
len(city_csv_file)

554

In [4]:
#type(city_csv_file)

In [5]:
#city_csv_file.columns

In [6]:
#Narrow down the cities to fit weather conditions
#Drop any rows will null values
#vacation_criteria_df = pd.DataFrame(city_data_df,  columns = ["City", "Country", "Max Temp", "Humidity", "Cloudiness"]
#new = old[['A', 'C', 'D']].copy().

vacation_criteria_df = city_csv_file[['city_name', 'country', 'temp', 'humidity', 'cloudiness', 'latitude', 'longitude' ]].copy()

vacation_city_df = vacation_criteria_df.loc[(vacation_criteria_df["cloudiness"]== 0) & (vacation_criteria_df["temp"] >= 70) & (vacation_criteria_df["temp"] <= 80)].dropna()

#vacation_city_df

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
len(vacation_city_df)

28

In [8]:
vacation_city_df = vacation_city_df[['city_name', 'country', 'temp', 'humidity', 'cloudiness', 'latitude', 'longitude']]

vacation_city_sort_df = vacation_city_df.sort_values(by = ['country', 'city_name'])

#Sorted by Country
#vacation_city_sort_df 

In [9]:
len(vacation_city_sort_df)

28

In [10]:
#vacation_city_sort_df.columns

### LOAD GOOGLE MAP¶



In [11]:
locations = vacation_city_sort_df[["latitude", "longitude"]]

#locations

In [12]:
gmaps.configure(api_key=g_key)

In [13]:
# locations
humidity = vacation_city_sort_df["humidity"].astype(float)

# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:

# locations
humidity = vacation_city_sort_df["humidity"].astype(float)
# humidity
figure_layout = {
    'width' : '1000px',
    'height':'400px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = vacation_city_sort_df

hotel_df["Hotel Name"] = ""

hotel_df = hotel_df.reset_index(drop=True)

#hotel_df

In [20]:
#creates a list to capture hotel_names to insert into dataframe
Hotel_Name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

In [24]:
# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df  
    lat = row["latitude"]
    lng = row["longitude"]
    city_name = row["city_name"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
#Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 0: Broken Hill.
Closest hotel in Broken Hill is Royal Exchange Hotel.
--------------------Ending Search
Retrieving Results for Index 1: Busselton.
Closest hotel in Busselton is The Sebel Busselton.
--------------------Ending Search
Retrieving Results for Index 2: Mackay.
Closest hotel in Mackay is Ocean International Hotel.
--------------------Ending Search
Retrieving Results for Index 3: Qena.
Closest hotel in Qena is Basma Hotel.
--------------------Ending Search
Retrieving Results for Index 4: Almeria.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 5: Lalibela.
Closest hotel in Lalibela is Maribela Hotel Lalibela.
--------------------Ending Search
Retrieving Results for Index 6: Amapala.
Closest hotel in Amapala is Cabañas Villas Ruta del Sol.
--------------------Ending Search
Retrieving Results for Index 7: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
--------------------Ending Se

In [25]:
hotel_df

,city_name,country,temp,humidity,cloudiness,latitude,longitude,Hotel Name
0,Broken Hill,AU,73.40,27,0,-31.9500,141.4333,Royal Exchange Hotel
1,Busselton,AU,78.01,64,0,-33.6500,115.3333,The Sebel Busselton
2,Mackay,AU,78.80,73,0,-21.1500,149.2000,Ocean International Hotel
3,Qena,EG,78.80,27,0,26.1615,32.7183,Basma Hotel
4,Almeria,ES,72.00,67,0,37.1667,-2.3333,
5,Lalibela,ET,77.61,17,0,12.0333,39.0333,Maribela Hotel Lalibela
6,Amapala,HN,76.68,59,0,13.2922,-87.6539,Cabañas Villas Ruta del Sol
7,Kiryat Gat,IL,78.01,41,0,31.6100,34.7642,Desert Gat
8,Chas,IN,70.50,23,0,23.6333,86.1667,Hotel Reliance
9,Dhemāji,IN,71.60,73,0,27.4833,94.5833,"Hotel The Imperial Inn, Dhemaji"


In [26]:
len(hotel_df)

28

In [27]:
#name_address

In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]


In [33]:
#hotel_info

In [34]:
target_coordinates = f"{hotel_df['latitude'][0]},{hotel_df['longitude'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [35]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [ ]:
example
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["city_name"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")